In [1]:
# Imports
import argparse
import os
import sys
import time
import glob
import re

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
from _datetime import datetime
import scipy.stats as stats
import matplotlib.animation as animation
from matplotlib import rcParams
from PIL import Image
from sklearn.utils import resample
from scipy.stats import pearsonr
import scipy.stats as stats
import scipy

In [2]:
# Local imports
sys.path.append("/home/users/benhutch/skill-maps")
import dictionaries as dicts

sys.path.append("/home/users/benhutch/skill-maps/python")
import functions as fnc

In [3]:
# Imports of historical functions
#/home/users/benhutch/skill-maps-differences/functions.py
import os

path = "/home/users/benhutch/skill-maps-differences"
# print(os.listdir(path))

# append path and import
sys.path.append(path)
import functions_diff as hist_fnc

# # List the functions in the module
# print(dir(hist_fnc))

In [4]:
# Set up the parameters
# Test run with psl dcppA-hindcast raw data and historical data
variable = "tas"
obs_var_name = "tas"
region = "global"
region_grid = dicts.gridspec_global
forecast_range = "2-9"
season = "DJFM"
model_season = "DJFM"
start_year = "1960"
end_year = "2022"
no_bootstraps = 1000

# Set up the paths
observations_path = dicts.obs
dcpp_models = dicts.common_models_noCMCC
historical_models = dicts.rsds_models_historical
base_dir = dicts.base_dir
base_dir_historical = dicts.base_dir_historical
output_dir = dicts.plots_dir
save_dir = dicts.save_dir

In [5]:
%tb
# Process the observations
obs = fnc.process_observations(variable, region, region_grid, forecast_range, season,
                                observations_path, obs_var_name, plev=None)

No traceback available to show.


Gridspec file: /home/users/benhutch/gridspec/gridspec-global.txt
Variable is not ua or va, creating new file name
File already exists
Loading ERA5 data
Dataset loaded:  [[[[244.69833 244.69833 244.69833 ... 244.69833 244.69833 244.69833]
   [243.98952 244.31924 244.6137  ... 243.28072 243.4782  243.7462 ]
   [251.60124 254.06972 254.60045 ... 251.7088  251.54306 251.9345 ]
   ...
   [247.25496 247.31139 247.20207 ... 246.92172 247.02046 247.10509]
   [246.12299 246.09302 246.05775 ... 246.36102 246.27638 246.24112]
   [245.3895  245.33836 245.29428 ... 245.57993 245.5235  245.45827]]

  [[      nan       nan       nan ...       nan       nan       nan]
   [      nan       nan       nan ...       nan       nan       nan]
   [      nan       nan       nan ...       nan       nan       nan]
   ...
   [      nan       nan       nan ...       nan       nan       nan]
   [      nan       nan       nan ...       nan       nan       nan]
   [      nan       nan       nan ...       nan       na

In [6]:
obs

<xarray.DataArray 't2m' (time: 65, lat: 72, lon: 144)>
dask.array<truediv, shape=(65, 72, 144), dtype=float64, chunksize=(8, 72, 100), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1959-12-31 1960-12-31 ... 2023-12-31
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
  * lat      (lat) float64 -90.0 -87.5 -85.0 -82.5 -80.0 ... 80.0 82.5 85.0 87.5

In [7]:
if variable in ['rsds', 'ssrd']:
    print("converting from J/m2 to W/m2")
    obs = obs / 86400

In [8]:
# Load the historical data
hist_data = hist_fnc.load_processed_historical_data(base_dir_historical, historical_models,
                                                    variable, region, forecast_range, season)

# Extract the data for the variable from the historical data
hist_data, _ = hist_fnc.extract_historical_data(hist_data, variable)

processing model:  BCC-CSM2-MR
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/tas/BCC-CSM2-MR/global/years_2-9/DJFM/outputs/processed/*.nc


processing model:  MPI-ESM1-2-HR
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/tas/MPI-ESM1-2-HR/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  CanESM5
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/tas/CanESM5/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  CMCC-CM2-SR5
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/tas/CMCC-CM2-SR5/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  HadGEM3-GC31-MM
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/tas/HadGEM3-GC31-MM/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  EC-Earth3
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/tas/EC-Earth3/global/years_2-9/DJFM/outputs/processed/*.nc
processing model:  MPI-ESM1-2-LR
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/tas/MPI-ESM1-2-LR/global/years_2-9/DJFM/outputs/processed/*.nc
processing model

In [9]:
# Load the DCPP data
dcpp_data = fnc.load_data(base_dir, dcpp_models, variable, region, 
                            forecast_range, model_season, level=None)

# Extract the data for the variable from the DCPP data
dcpp_data, _ = fnc.process_data(dcpp_data, variable)

Skipping file /home/users/benhutch/skill-maps-processed-data/tas/EC-Earth3/global/years_2-9/DJFM/outputs/mergetime/mergetime_EC-Earth3_tas_global_2-9_DJFM-r9i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/tas/EC-Earth3/global/years_2-9/DJFM/outputs/mergetime/mergetime_EC-Earth3_tas_global_2-9_DJFM-r8i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/tas/EC-Earth3/global/years_2-9/DJFM/outputs/mergetime/mergetime_EC-Earth3_tas_global_2-9_DJFM-r6i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/tas/EC-Earth3/global/years_2-9/DJFM/outputs/mergetime/mergetime_EC-Earth3_tas_global_2-9_DJFM-r7i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/tas/EC-Earth3/global/years_2-9/DJFM/outputs/mergetime/mergetime_EC-Earth3_tas_global_2-9_DJFM-r10i2.nc
Skipping file /home/users/benhutch/skill-maps-processed-data/tas/FGOALS-f3-L/global/years_2-9/DJFM/outputs/mergetime/mergetime_FGOALS-f3-L_tas_global_2-9_DJFM-r1i1.nc
Skipping file /ho

In [10]:
# Now we want to make sure that the time period 
# Use constrain years to make sure that the model data is the same
constrained_hist_data = fnc.constrain_years(hist_data, historical_models)

# Constrain the years for the DCPP data
constrained_dcpp_data = fnc.constrain_years(dcpp_data, dcpp_models)

# Align the forecasts and observations
fcst1, fcst2, obs_array, common_years = fnc.align_forecast1_forecast2_obs(constrained_dcpp_data, dcpp_models, 
                                                                    constrained_hist_data, historical_models, 
                                                                    obs)

There is a gap of more than 1 year in the years for model CanESM5member r1i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r13i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r11i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r16i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r18i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r20i1p2f1
There is a gap of more than 1 year in the years for model CanESM5member r1i1p2f1
adjusted indices: 54
years where gap is greater than 1: [2019 2020 2021 2023 2024]
There is a gap of more than 1 year in the years for model CanESM5member r13i1p2f1
adjusted indices: 54
years where gap is greater than 1: [2019 2020 2021 2023 2024]
There is a gap of more than 1 year in the years for model CanESM5member r11i1p2f1
adjusted indices: 54
years where gap is greater than 1: [2019 2020 2021 2023 2024]
There is a gap 

In [11]:
print(constrained_hist_data[historical_models[0]])

[<xarray.DataArray 'tas' (time: 38, lat: 72, lon: 144)>
dask.array<getitem, shape=(38, 72, 144), dtype=float32, chunksize=(38, 45, 45), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
  * lat      (lat) float64 -90.0 -87.5 -85.0 -82.5 -80.0 ... 80.0 82.5 85.0 87.5
    height   float64 ...
  * time     (time) object 1974-12-31 00:00:00 ... 2011-12-31 00:00:00
Attributes:
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    comment:        near-surface (usually, 2 meter) air temperature
    original_name:  TREFHT
    cell_methods:   area: time: mean (interval: 5 minutes)
    cell_measures:  area: areacella
    history:        2018-11-26T05:08:26Z altered by CMOR: Treated scalar dime..., <xarray.DataArray 'tas' (time: 38, lat: 72, lon: 144)>
dask.array<getitem, shape=(38, 72, 144), dtype=float32, chunksize=(38, 45, 45), chunktype=numpy.ndarray>
Coordinates:
  * lon 

In [12]:
# Now that we have the forecast1 array for the dcpp data
# We should be able to perform the lagging to quadruple the number of ensemble 
# members
# NOTE: Would it be worth lagging the historical members?
# Set up the lag
lag = 4

# extract the no_members
n_members = fcst1.shape[0] ; n_years = fcst1.shape[1]

# Extract the no_lats
n_lats = fcst1.shape[2] ; n_lons = fcst1.shape[3]

# Set up the no_lagged_members
n_lagged_members = n_members * lag

# Set up the lagged ensemble
lagged_ensemble = np.zeros([n_lagged_members, n_years, n_lats, n_lons])

# Loop over the ensemble members
for member in range(n_members):
    # Loop over the years
    for year in range(n_years):
        # If the year is less than the lag
        if year < lag - 1:
            # Set the lagged ensemble member equal to NaN
            lagged_ensemble[member, year, :, :] = np.nan

            # Also set the lagged ensemble member equal to NaN
            for lag_i in range(lag):
                lagged_ensemble[member + (lag_i * n_members), year, :, :] \
                = np.nan
        # Otherwise
        else:
            # Loop over the lag
            for lag_i in range(lag):
                # Set the lagged ensemble member equal to the forecast
                lagged_ensemble[member + (lag_i * n_members), year, :, :] = \
                    fcst1[member, year - lag_i, :, :]
                
# Now we have the lagged ensemble
# The first 3 years of the lagged ensemble should be NaN
# Remove these years
lagged_ensemble = lagged_ensemble[:, lag-1:, :, :]

# Do the same for the observations
lagged_obs = obs_array[lag-1:, :, :]

# Do the same for the second forecast
lagged_fcst2 = fcst2[:, lag-1:, :, :]

# Check the shape of the lagged ensemble
if lagged_ensemble[0].shape == lagged_obs.shape:
    print("lagged ensemble and observations have the same shape")
else:
    raise ValueError("lagged ensemble and observations do not have the same" +
                        "shape")

# Check the shape of the lagged ensemble
if lagged_fcst2[0].shape == lagged_obs.shape:
    print("lagged ensemble and observations have the same shape")
else:
    raise ValueError("lagged ensemble and observations do not have the same" +
                        "shape")

lagged ensemble and observations have the same shape
lagged ensemble and observations have the same shape


In [13]:
# Load up the data for the NAO matched members
base_dir_boot = "/gws/nopw/j04/canari/users/benhutch/NAO-matching"

# Set up the path to the data
path_nao_match_dir = f"{base_dir_boot}/" + \
                        f"{variable}/{region}/{season}/{forecast_range}/" + \
                            f"{start_year}-{end_year}/"

# Check if there are files in the directory
if len(os.listdir(path_nao_match_dir)) == 0:
    raise ValueError("There are no files in the directory")

# Extract the files in the directory
files = os.listdir(path_nao_match_dir)

print(files)

# Extract the file containing the lagged ensemble NAO matched mean
nao_matched_mean_file = [file for file in files if "mean_lagged" in file][0]

# Extract the file containing the lagged ensemble NAO matched members
nao_matched_members_file = [file for file in files if "members_lagged" in file][0]

# pprint the files
print("members file: ", nao_matched_members_file)
print("mean file: ", nao_matched_mean_file)

# Open the datasets using xarray
nao_matched_members = xr.open_dataset(path_nao_match_dir + 
                                        nao_matched_members_file)

nao_matched_mean = xr.open_dataset(path_nao_match_dir +
                                    nao_matched_mean_file)

['tas_global_DJFM_2-9_1960-2022_matched_var_ensemble_mean.nc', 'tas_global_DJFM_2-9_1960-2022_matched_var_ensemble_mean_lagged.nc', 'tas_global_DJFM_2-9_1960-2022_matched_var_ensemble_members.nc', 'tas_global_DJFM_2-9_1960-2022_matched_var_ensemble_members_lagged.nc']
members file:  tas_global_DJFM_2-9_1960-2022_matched_var_ensemble_members_lagged.nc
mean file:  tas_global_DJFM_2-9_1960-2022_matched_var_ensemble_mean_lagged.nc


In [14]:
# # Testing nao_matched_members
# # Extract the years
# nao_matched_members_years = nao_matched_members.time.values
# print(nao_matched_members_years)

# print(nao_matched_members)

# # Extract the data for a given year
# year = 1970
# nao_matched_members_year = nao_matched_members.sel(time=year)
# print(nao_matched_members_year)

In [15]:
# Now we want to extract the consistent years from the NAO matched members
# and convert to an array
# Assert that obs must be an xarray dataset
assert isinstance(obs, xr.DataArray), "obs must be an xarray DataArray"

# First extract the years for the observations
obs_years = obs.time.dt.year.values

# Loop over the years
for year in obs_years:
    # If there are any NaN values in the observations
    obs_year = obs.sel(time=f'{year}')
    if np.isnan(obs_year.values).any():
        print(f"there are NaN values in the observations for {year}")
        if np.isnan(obs_year.values).all():
            print(f"all values are NaN for {year}")
            # Delete the year from the observations
            obs = obs.sel(time=obs.time.dt.year != year)
        else:
            print(f"not all values are NaN for {year}")
    else:
        print(f"there are no NaN values in the observations for {year}")

# Extract the constrained obs_years
obs_years = obs.time.dt.year.values

# Extract the years for the NAO matched members
nao_matched_members_years = nao_matched_members.time.values


# Check that there are no duplicate years in the NAO matched members
if len(nao_matched_members_years) != len(np.unique(nao_matched_members_years)):
    raise ValueError("there are duplicate years in the NAO matched members")

# Loop over the years for the NAO matched members
# and check that there are no NaN values
for year in nao_matched_members_years:
    # If there are any NaN values in the observations
    nao_matched_year = nao_matched_members['__xarray_dataarray_variable__'].sel(time=year)
    if np.isnan(nao_matched_year['__xarray_dataarray_variable__'].
                    values).any():
        print(f"there are NaN values in the NAO matched members for {year}")
        if np.isnan(nao_matched_year['__xarray_dataarray_variable__'].
                        values).all():
            print(f"all values are NaN for {year}")
            # Delete the year from the observations
            nao_matched_members = nao_matched_members.sel(time=nao_matched_members.time.values != year)
        else:
            print(f"not all values are NaN for {year}")
    else:
        print(f"there are no NaN values in the NAO matched members for {year}")

# Extract the years for the constrained historical data
constrained_hist_data_years = constrained_hist_data[historical_models[0]][0]. \
time.dt.year.values

# If the years for the NAO matched members are not the same as the
# constrained historical data
if np.array_equal(nao_matched_members_years, 
                    constrained_hist_data_years) == False:
    print("years for NAO matched members and constrained historical data" +
            "are not the same")
    
    # Find the common years
    common_years = np.intersect1d(nao_matched_members_years, 
                                    constrained_hist_data_years)
    
    # Extract the common years from the NAO matched members
    nao_matched_members = nao_matched_members.sel(time=nao_matched_members.
                                                    time.dt.year.isin(common_years))
    
    constrained_hist_data_nmatch = {}

    # Extract the common years from the constrained historical data
    for model in historical_models:
        model_data = constrained_hist_data[model]
        for member in model_data:
            # Extract the years for the member
            member_years = member.time.dt.year.values

            # If the years for the member are not the same as the common years
            if np.array_equal(member_years, common_years) == False:
                print(f"years for {model} member {member} are not the same" +
                        "as the common years")
                # Extract the common years for the member
                member = member.sel(time=member.time.dt.year.isin(common_years))
            
            # Append to the list
            if model not in constrained_hist_data_nmatch:
                constrained_hist_data_nmatch[model] = []

            constrained_hist_data_nmatch[model].append(member)

    # Check the new years
    nao_matched_years = nao_matched_members.time.dt.year.values

    # Check the new years for the constrained historical data
    constrained_hist_data_nmatch_years = constrained_hist_data_nmatch[historical_models[0]][0]. \
    time.dt.year.values

    # Assert that the arrays are the same
    assert np.array_equal(nao_matched_years, 
                            constrained_hist_data_nmatch_years) == True, \
                                "the years are not the same"
    

# If the forecast1 and forecast2 are now the same
common_f_years = nao_matched_years

# If the obs_years and common_f_years are not the same
if np.array_equal(obs_years, common_f_years) == False:
    print("obs_years and common_f_years are not the same")
    # Find the common years
    common_years = np.intersect1d(obs_years, common_f_years)
    
    # Extract the common years from the observations
    obs = obs.sel(time=obs.time.dt.year.isin(common_years))
    
    # Extract the common years from the NAO matched members
    nao_matched_members = nao_matched_members.sel(time=nao_matched_members.
                                                    time.dt.year.isin(common_years))
    
    constrained_hist_data_nmatch_obs = {}

    # Extract the common years from the constrained historical data
    for model in historical_models:
        model_data = constrained_hist_data[model]
        for member in model_data:
            # Extract the years for the member
            member_years = member.time.dt.year.values

            # If the years for the member are not the same as the common years
            if np.array_equal(member_years, common_years) == False:
                print(f"years for {model} member {member} are not the same" +
                        "as the common years")
                # Extract the common years for the member
                member = member.sel(time=member.time.dt.year.isin(common_years))
            
            # Append to the list
            if model not in constrained_hist_data_nmatch_obs:
                constrained_hist_data_nmatch_obs[model] = []

            constrained_hist_data_nmatch_obs[model].append(member)

    # Check the new years
    obs_years = obs.time.dt.year.values

    # Check the new years for the constrained historical data
    constrained_hist_data_nmatch_years = constrained_hist_data_nmatch_obs[historical_models[0]][0]. \
    time.dt.year.values

    # Assert that the arrays are the same
    assert np.array_equal(obs_years, 
                            constrained_hist_data_nmatch_years) == True, \
                                "the years are not the same"

    # Extract the nao_matched_years again
    nao_matched_years = nao_matched_members.time.dt.year.values

    # Assert that the arrays are the same
    assert np.array_equal(obs_years, 
                            nao_matched_years) == True, \
                                "the years are not the same"
    
# Extract the arrays from the datasets
nao_matched_members = nao_matched_members['__xarray_dataarray_variable__'].values

# Extract the obs
obs = obs.values

# Extract the no ensemble members for f2 (historical data)
n_members_hist = np.sum([len(constrained_hist_data_nmatch_obs[model]) for model in constrained_hist_data_nmatch_obs])

# Create an empty array for the historical data
hist_data_array = np.zeros([n_members_hist, len(obs_years), n_lats, n_lons])

# Initialise the counter
member_index = 0

# Loop over the models
for model in historical_models:
    # Extract the model data
    model_data = constrained_hist_data_nmatch_obs[model]

    # Loop over the members
    for member in model_data:
        # Extract the member data
        member_data = member.values

        # Add to the array
        hist_data_array[member_index, :, :, :] = member_data

        # Add to the counter
        member_index += 1

# Print the shape of the array
print("shape of the historical data array: ", hist_data_array.shape)
print("shape of the NAO matched members: ", nao_matched_members.shape)
print("shape of the observations: ", obs.shape)

# Extract the first ensemble member
nao_matched_members_ens1 = nao_matched_members[0]
hist_data_array_ens1 = hist_data_array[0]

# Assert that the arrays are the same
assert np.array_equal(nao_matched_members_ens1, hist_data_array_ens1) == True, \
    "the arrays are not the same"

# Assert that the arrays are the same as the obs
assert np.array_equal(nao_matched_members_ens1, obs) == True, \
    "the arrays are not the same"


KeyError: 1959

In [ ]:
# Now we can test the bootstrapping function
forecast_stats_raw = fnc.forecast_stats(obs_array, fcst1, fcst2, no_boot=10)

bootstrap index 0
shape of obs_boot (45, 72, 144)
value of obs_boot [[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 5.66360532  6.20386429  6.12671369 ...  6.07562066  5.94156901
    6.00949978]
  [ 4.24652705  3.93631149  3.89064417 ...  4.29845414  4.4922866
    4.49233073]
  [ 2.68602051  2.71219455  2.72397045 ...  2.43226002  2.34866211
    2.51464699]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 3.41969907  4.10251302  4.26025174 ...  3.71201534  3.65585974
    3.74875217]
  [ 2.03901259  1.62503816  1.45576136 ...  2.10778139  2.35274161
    2.34255986]
  [-0.79607883 -0.70073134 -0.63121139 ... -1.08111744

In [ ]:
# process the forecast stats for the lagged ensemble
forecast_stats_lagged = fnc.forecast_stats(lagged_obs, lagged_ensemble, 
                                            lagged_fcst2, no_boot=10)

{'corr1': array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [0.51331272, 0.54403432, 0.54812514, ..., 0.58840733, 0.54480468,
        0.52989605],
       [0.30633439, 0.27842244, 0.29847085, ..., 0.34387112, 0.34059382,
        0.33245234],
       [0.22729118, 0.23861865, 0.24867063, ..., 0.19272982, 0.17833131,
        0.20252902]]), 'corr1_min': array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ...,
       [ 0.18348989,  0.18211674,  0.15375806, ...,  0.25829093,
         0.20143739,  